In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVC

import sys

##########################################
#### CHANGE THIS PARAMETERS
##########################################

OMICA = 'meth'  # se si vuole fare preproc su un solo dataset
FEAT_SELECTION_METHOD = 'PCA'   # per ablation study
                                  # Choose among PCA, KBest, ensemble

# same number of features as in the paper

if FEAT_SELECTION_METHOD=='KBest':
  num_features_per_omic = {'mRNA' : 3217,
                          'miRNA' : 383,
                          'meth' : 3139}
elif FEAT_SELECTION_METHOD=='PCA':
  num_features_per_omic = {'mRNA' : 1107, # for PCA maximum features selezionabili = 1107 (lunghezza miRNA df)
                          'miRNA' : 383,
                          'meth' : 1119} # for PCA maximum features selezionabili = 1119 (lunghezza meth df)
elif FEAT_SELECTION_METHOD=='ensemble':
  num_features_per_omic = {'mRNA' : 10000, 
                          'miRNA' : 500,
                          'meth' : 20000} 

# subtype mapping
dict_label = {'TCGA-LUAD0' : 0,
              'TCGA-LUAD1' : 1,
              'TCGA-LUSC0' : 2,
              'TCGA-LUSC1' : 3}

In [ ]:
def toString(s):
  return str(s)

def map2Class(r):
  return dict_label[r]

def normalize(df):
  # Normalize dataframe
  scaler = StandardScaler()
  mat = scaler.fit_transform(df.values.astype(float))
  df.iloc[::, ::] = mat
  # print(df.shape)
  return df

def preproc(data,omica):
  print(f">>> Preprocessing {omica}...")

  # separate label from numerical data
  label_info = data.iloc[:,-3:]
  numerical_data = data.iloc[:,:-3]

  # preprocess label
  case_ids = label_info['case_id']
  label_info = label_info['subtype'] + label_info['label'].apply(toString)
  # map subtype labels to integer
  subtype = label_info.apply(map2Class)

  # preprocess numerical data
  # apply log2 to miRNA datasets 
  if 'miRNA' in omica or 'mRNA' in omica:
    numerical_data = np.log2(numerical_data + 1)
  elif 'meth' in omica:
    # remove column with mored than 50% of nan values
    print(f">>> Removing columns with 0.3% of nans in {omica}...")
    # get name columns 
    column_names = numerical_data.columns# [:-1] # eccetto la colonna della label
    cnt_cc=0
    cnt_nan=0

    # for each colomn
    for cname in column_names:
      dropped = False 
      # get current column 
      curr_col = numerical_data[cname]
      
      # remove column with mored than 50% of nan values
      if curr_col.isna().sum() > 0.3*len(numerical_data): 
        numerical_data = numerical_data.drop(columns=[cname])
        cnt_nan+=1
        dropped = True

      # remove costant column 
      std = curr_col.std()
      if std == 0 :
        numerical_data = numerical_data.drop(columns=[cname])
        cnt_cc+=1
        dropped = True

      if not dropped:
        # fill na with the mean() on the column
        numerical_data[cname] = curr_col.interpolate()

    print(f"Removed costant columns = {cnt_cc}")
    print(f"Removed NaN (> 50%) columns = {cnt_nan}\n\n")
    
    # fill nan values
    if numerical_data.isna().sum().sum() > 0: 
      numerical_data.fillna(0,inplace=True) #(method='backfill',axis=1,inplace=True)

  # normalize numerical data 
  df_norm = normalize(numerical_data)

  # select numerical data 
  ##############################################################################
  # 1) Select K Best
  n_elements = num_features_per_omic[omica] # value from paper Subtype-GAN
  print(f">>> Keeping only the best k = {n_elements} columns")

  if FEAT_SELECTION_METHOD=='PCA':
    ipca = IncrementalPCA(n_components=n_elements)
    numerical_data_selected = pd.DataFrame(ipca.fit_transform(df_norm, label_info))

  elif FEAT_SELECTION_METHOD=='KBest':
    selKbest = SelectKBest(score_func=f_classif, k=n_elements)
    kbest = selKbest.fit_transform(df_norm, label_info)
    mask = selKbest.get_support() #list of booleans

    new_features = [] # The list of your K best features
    feature_names = df_norm.columns.values

    for boool, feature in zip(mask, feature_names):
        if boool:
            new_features.append(feature)
  
    # keep only numerical data selected
    numerical_data_selected = pd.DataFrame(kbest, columns=new_features)
  elif FEAT_SELECTION_METHOD=='ensemble':
    # apply KBest
    selKbest = SelectKBest(score_func=f_classif, k=n_elements)
    kbest = selKbest.fit_transform(df_norm, label_info)
    mask = selKbest.get_support() #list of booleans

    new_features_KBest = [] # The list of your K best features
    feature_names = df_norm.columns.values

    for boool, feature in zip(mask, feature_names):
        if boool:
            new_features_KBest.append(feature)

    # Apply Random forests
    #X_train,X_test,y_train,y_test = train_test_split(df_norm,subtype,random_state=42)
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 300, random_state=7))
    sel.fit(df_norm, subtype)
    new_features_RF = df_norm.columns[(sel.get_support())].tolist()

    # Apply LinearSVC
    lsvc = LinearSVC(C=0.1, penalty="l1", dual=False, random_state=32)
    sel = SelectFromModel(lsvc)
    sel.fit(df_norm, subtype)
    new_features_SVC = df_norm.columns[(sel.get_support())].tolist()
    
    final_features = []
    for f in feature_names:
      count_f = 0
      if f in new_features_KBest:
        count_f += 1
      if f in new_features_RF:
        count_f += 1
      if f in new_features_SVC:
        count_f += 1
      if count_f >= 2:
        final_features.append(f)

    print(final_features)
    print(len(final_features))

    # keep only numerical data selected
    numerical_data_selected = df_norm[final_features]

  ############################################################################

  # df concat: numerical | case_id | subtype
  df = pd.concat([numerical_data_selected, case_ids, subtype], axis=1)
  # rename label column 
  df.columns = [*df.columns[:-1], 'subtype']

  return df

In [ ]:
for omica in ['miRNA','mRNA','meth']: 
  for FEAT_SELECTION_METHOD in ['KBest','PCA','ensemble']:

    if FEAT_SELECTION_METHOD=='KBest':
      num_features_per_omic = {'mRNA' : 3217,
                              'miRNA' : 383,
                              'meth' : 3139}
    elif FEAT_SELECTION_METHOD=='PCA':
      num_features_per_omic = {'mRNA' : 1107, # for PCA maximum features selezionabili = 1107 (lunghezza miRNA df)
                              'miRNA' : 383,
                              'meth' : 1119} # for PCA maximum features selezionabili = 1119 (lunghezza meth df)
    elif FEAT_SELECTION_METHOD=='ensemble':
      num_features_per_omic = {'mRNA' : 10000, 
                              'miRNA' : 500,
                              'meth' : 20000} 

    # file input
    print(f">>> Reading {omica}...")
    input_file = f"/content/drive/MyDrive/Bioinformatics/datasets/non toccare/{omica}_full_dataset.csv"

    # read single dataset
    df = pd.read_csv(input_file)

    # apply preprocessing
    df_preproc = preproc(df,omica)
    
    # dump to file 
    out_file = f"/content/drive/MyDrive/Bioinformatics/datasets/{omica}_{FEAT_SELECTION_METHOD}_edited_dataset.csv"
    df_preproc.to_csv(out_file,header=True,index=False,sep=',')
    print(f">>> File saved in: {out_file}\n")


>>> Reading miRNA...
>>> Preprocessing miRNA...
>>> Keeping only the best k = 383 columns


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  18   19   32   41   42   45   48   57   58   69   70   84   85   95
  102  112  113  114  121  150  151  152  153  155  156  157  158  159
  160  169  172  196  259  260  277  278  279  333  340  347  348  363
  364  365  366  372  384  435  436  437  438  467  481  525  526  546
  550  551  552  553  554  569  570  577  582  586  587  619  660  662
  663  664  674  675  676  678  682  684  685  688  689  690  691  692
  695  696  697  698  700  704  705  706  707  709  711  715  716  718
  719  722  723  724  726  728  729  730  732  734  737  739  740  743
  745  746  753  756  757  759  762  796  804  805  821  822  825  827
  828  833  838  847  851  855  864  865  866  874  889  896  907 1086
 1145 1186 1222 1227 1228 1233 1234 1259 1260 1263 1264 1265 1267 1269
 1270 1271 1272 1290 1300 1304 1324 1327 1329 1340 1345 1357 1358 1359
 1370 1376 1405 1407 1409 1411 

>>> File saved in: /content/drive/MyDrive/Bioinformatics/datasets/miRNA_KBest_feat_sel_dataset.csv

>>> Reading miRNA...
>>> Preprocessing miRNA...
>>> Keeping only the best k = 383 columns
>>> File saved in: /content/drive/MyDrive/Bioinformatics/datasets/miRNA_PCA_feat_sel_dataset.csv

>>> Reading miRNA...
>>> Preprocessing miRNA...
>>> Keeping only the best k = 500 columns


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  18   19   32   41   42   45   48   57   58   69   70   84   85   95
  102  112  113  114  121  150  151  152  153  155  156  157  158  159
  160  169  172  196  259  260  277  278  279  333  340  347  348  363
  364  365  366  372  384  435  436  437  438  467  481  525  526  546
  550  551  552  553  554  569  570  577  582  586  587  619  660  662
  663  664  674  675  676  678  682  684  685  688  689  690  691  692
  695  696  697  698  700  704  705  706  707  709  711  715  716  718
  719  722  723  724  726  728  729  730  732  734  737  739  740  743
  745  746  753  756  757  759  762  796  804  805  821  822  825  827
  828  833  838  847  851  855  864  865  866  874  889  896  907 1086
 1145 1186 1222 1227 1228 1233 1234 1259 1260 1263 1264 1265 1267 1269
 1270 1271 1272 1290 1300 1304 1324 1327 1329 1340 1345 1357 1358 1359
 1370 1376 1405 1407 1409 1411 

['hsa-let-7a-1', 'hsa-let-7a-2', 'hsa-let-7a-3', 'hsa-let-7b', 'hsa-let-7c', 'hsa-let-7d', 'hsa-let-7e', 'hsa-let-7f-1', 'hsa-let-7f-2', 'hsa-let-7g', 'hsa-let-7i', 'hsa-mir-1-1', 'hsa-mir-1-2', 'hsa-mir-100', 'hsa-mir-101-1', 'hsa-mir-101-2', 'hsa-mir-103a-1', 'hsa-mir-103a-2', 'hsa-mir-105-1', 'hsa-mir-105-2', 'hsa-mir-106a', 'hsa-mir-107', 'hsa-mir-10a', 'hsa-mir-1245a', 'hsa-mir-1246', 'hsa-mir-1247', 'hsa-mir-1248', 'hsa-mir-1254-1', 'hsa-mir-1258', 'hsa-mir-125a', 'hsa-mir-126', 'hsa-mir-1266', 'hsa-mir-1269a', 'hsa-mir-1270', 'hsa-mir-1271', 'hsa-mir-1275', 'hsa-mir-1287', 'hsa-mir-1291', 'hsa-mir-1293', 'hsa-mir-1295a', 'hsa-mir-1296', 'hsa-mir-1301', 'hsa-mir-1307', 'hsa-mir-130a', 'hsa-mir-130b', 'hsa-mir-132', 'hsa-mir-133a-1', 'hsa-mir-133a-2', 'hsa-mir-133b', 'hsa-mir-134', 'hsa-mir-135a-1', 'hsa-mir-135b', 'hsa-mir-136', 'hsa-mir-137', 'hsa-mir-138-1', 'hsa-mir-138-2', 'hsa-mir-139', 'hsa-mir-140', 'hsa-mir-141', 'hsa-mir-142', 'hsa-mir-143', 'hsa-mir-144', 'hsa-mir-145',

...

['ENSG00000172137.17', 'ENSG00000070087.12', 'ENSG00000153561.11', 'ENSG00000070081.14', 'ENSG00000166634.5', 'ENSG00000276644.3', 'ENSG00000064225.11', 'ENSG00000167768.4', 'ENSG00000218358.2', 'ENSG00000036448.8', 'ENSG00000152942.17', 'ENSG00000169992.8', 'ENSG00000127564.15', 'ENSG00000255521.1', 'ENSG00000138669.8', 'ENSG00000277670.1', 'ENSG00000150773.9', 'ENSG00000205362.9', 'ENSG00000109805.8', 'ENSG00000221923.7', 'ENSG00000163975.10', 'ENSG00000239893.1', 'ENSG00000135723.12', 'ENSG00000168300.12', 'ENSG00000117394.18', 'ENSG00000164266.9', 'ENSG00000280106.1', 'ENSG00000123364.4', 'ENSG00000175309.13', 'ENSG00000011052.20', 'ENSG00000181649.5', 'ENSG00000100842.11', 'ENSG00000082068.7', 'ENSG00000165312.6', 'ENSG00000181991.14', 'ENSG00000166896.6', 'ENSG00000118094.10', 'ENSG00000168306.11', 'ENSG00000064102.13', 'ENSG00000268620.1', 'ENSG00000172345.12', 'ENSG00000057149.13', 'ENSG00000137440.4', 'ENSG00000196748.8', 'ENSG00000149925.15', 'ENSG00000054983.15', 'ENSG000002

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_incremental_pca.py:309: RuntimeWarning: Mean of empty slice.
  explained_variance[self.n_components_:].mean()
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


>>> File saved in: /content/drive/MyDrive/Bioinformatics/datasets/meth_PCA_feat_sel_dataset.csv

>>> Reading meth...
>>> Preprocessing meth...
>>> Removing columns with 0.3% of nans in meth...
Removed costant columns = 0
Removed NaN (> 50%) columns = 2598


>>> Keeping only the best k = 20000 columns


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


['cg00005847', 'cg00009407', 'cg00021527', 'cg00024812', 'cg00031162', 'cg00055233', 'cg00061629', 'cg00066816', 'cg00072216', 'cg00076645', 'cg00084687', 'cg00117172', 'cg00121158', 'cg00134787', 'cg00155167', 'cg00167504', 'cg00185839', 'cg00187686', 'cg00197381', 'cg00200063', 'cg00208830', 'cg00213331', 'cg00215066', 'cg00223186', 'cg00230368', 'cg00231140', 'cg00239071', 'cg00269115', 'cg00290028', 'cg00292971', 'cg00323915', 'cg00324733', 'cg00327185', 'cg00327483', 'cg00328227', 'cg00346145', 'cg00392257', 'cg00404599', 'cg00411097', 'cg00417297', 'cg00422913', 'cg00436282', 'cg00446235', 'cg00449941', 'cg00466249', 'cg00466544', 'cg00474209', 'cg00485380', 'cg00491404', 'cg00510956', 'cg00532335', 'cg00548268', 'cg00568128', 'cg00581156', 'cg00598858', 'cg00601486', 'cg00609097', 'cg00616129', 'cg00619207', 'cg00630583', 'cg00631230', 'cg00653387', 'cg00657095', 'cg00691625', 'cg00718513', 'cg00746130', 'cg00767581', 'cg00783759', 'cg00792740', 'cg00833393', 'cg00839584', 'cg00